In [ ]:
import pandas as pd
import json
import numpy as np

# read data

In [ ]:
book_path = 'Books.jsonl'
chunks = pd.read_json(book_path,lines = True, chunksize=10000)

In [ ]:
book_df = pd.concat(chunks)

In [ ]:
meta_path = 'meta_Books.jsonl'
meta_chunks = pd.read_json(meta_path,lines = True, chunksize=10000)

In [ ]:
meta_df = pd.concat(meta_chunks)

In [ ]:
print(book_df.columns)
print(meta_df.columns)

In [ ]:
book_small_df = book_df.copy()[['rating',  'parent_asin', 'user_id', 'timestamp']]

In [ ]:
book_small_df.drop_duplicates(inplace=True)

In [ ]:
book_small_df.to_hdf('./small_Books.hdf',key = '1')

In [ ]:
book_small_df.to_pickle('./small_Books.pkl')

In [ ]:
meta_small_df = meta_df.copy()[[ 'title', 'subtitle', 'author', 'average_rating',
       'rating_number', 'features',  'price', 
       'store', 'categories', 'details', 'parent_asin', ]]

# process item publish time

In [ ]:
import re
import datetime
def get_publication_time(details):
    if details == {}:
        return None
    else:
        keys = list(details.keys())
        if 'Publisher' in keys:
            date_str = details['Publisher']
            date_match = re.search(r'\((.*?)\)', date_str)
            if date_match:
                date_str = date_match.group(1)  

            try:
                date_obj = datetime.datetime.strptime(date_str, '%B %d, %Y')
                return date_obj.strftime('%Y-%m-%d')  
            except ValueError:
                return None  
        elif 'Publication Date' in keys:
            date_str = details['Publication Date']
            try:
                date_obj = datetime.datetime.strptime(date_str, '%B %d, %Y')
                return date_obj.strftime('%Y-%m-%d')  
            except ValueError:
                return None 

In [ ]:
meta_small_df['publish_time'] = pd.to_datetime(meta_small_df['details'].apply(get_publication_time), errors='coerce')

In [ ]:
meta_small_df.to_pickle('./small_meta_Books.pkl')

In [ ]:
print(meta_small_df['title'].unique().shape)

In [ ]:
print(meta_small_df['title'])